## 1. Preprocressing

In [7]:
import numpy as np
from datascience import *

In [9]:
daily_casualties_gaza = Table.read_table('casualties_daily_gaza_2025-04-12.csv')
daily_casualties_gaza.show(5)

report_date,report_source,report_period,ext_massacres_cum,killed,killed_cum,ext_killed,ext_killed_cum,ext_killed_children_cum,ext_killed_women_cum,injured_cum,ext_injured,ext_injured_cum,ext_civdef_killed_cum,med_killed_cum,ext_med_killed_cum,press_killed_cum,ext_press_killed_cum
2023-10-07,mohtel,24,0,232,232,232,232,0,0,1610,1610,1610,0,6,6,1,1
2023-10-08,mohtel,24,0,138,370,138,370,78,41,1788,178,1788,0,nan,6,1,1
2023-10-09,mohtel,24,8,190,560,190,560,91,61,2271,483,2271,0,6,6,3,3
2023-10-10,mohtel,24,8,340,900,340,900,260,230,4000,1729,4000,0,nan,6,7,7
2023-10-11,gmotel,24,23,200,1100,200,1100,398,230,5184,1184,5184,0,10,10,nan,7


In [11]:
daily_casualties_westbank = Table.read_table('casualties_daily_west_bank_2025-04-12.csv')
daily_casualties_westbank.show(5)

report_date,verified.killed,verified.killed_cum,verified.injured,verified.injured_cum,verified.killed_children,verified.killed_children_cum,verified.injured_children,verified.injured_children_cum,killed_cum,killed_children_cum,injured_cum,injured_children_cum,settler_attacks_cum,flash_source
2023-10-07,6,6,197,197,2,2,31,31,3,0,23,1,3,un
2023-10-08,8,14,127,324,3,5,7,38,13,1,194,1,9,un
2023-10-09,4,18,88,412,0,5,19,57,15,3,295,34,19,un
2023-10-10,5,23,72,484,0,5,7,64,19,3,332,39,29,un
2023-10-11,6,29,85,569,2,7,17,81,26,3,427,62,39,un


In [ ]:
pip install pdfplumber

In [1]:
import pdfplumber
from datascience import Table
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger("pdfminer").setLevel(logging.ERROR)

pdf_path = 'killed-in-gaza_moh_2025-03-23.pdf'

with pdfplumber.open(pdf_path) as pdf:
    total_pages = len(pdf.pages)
    batch_size = 50  # process 50 pages at a time
    batch_number = 1
    
    for start_page in range(0, total_pages, batch_size):
        end_page = min(start_page + batch_size, total_pages)
        print(f"Processing pages {start_page+1} to {end_page}...")

        batch_rows = []
        columns = None

        for page_num in range(start_page, end_page):
            page = pdf.pages[page_num]
            table = page.extract_table()
            if table:
                if columns is None:
                    columns = table[0]  # first page -> header
                    batch_rows.extend(table[1:])  # first page -> skip header
                else:
                    batch_rows.extend(table)  # other pages -> take all rows

        if batch_rows:
            # Building a small Table for this batch
            batch_table = Table().with_columns(*[(col, []) for col in columns])

            for row in batch_rows:
                batch_table = batch_table.with_row(row)

            # Saving this batch separately
            filename = f'killed_in_gaza_batch_{batch_number}.csv'
            batch_table.to_csv(filename)

        batch_number += 1

Processing pages 1 to 50...
Processing pages 51 to 100...
Processing pages 101 to 150...
Processing pages 151 to 200...
Processing pages 201 to 250...
Processing pages 251 to 300...
Processing pages 301 to 350...
Processing pages 351 to 400...
Processing pages 401 to 450...
Processing pages 451 to 500...
Processing pages 501 to 550...
Processing pages 551 to 600...
Processing pages 601 to 650...
Processing pages 651 to 700...
Processing pages 701 to 750...
Processing pages 751 to 800...
Processing pages 801 to 850...
Processing pages 851 to 900...
Processing pages 901 to 950...
Processing pages 951 to 981...


In [44]:
import csv
import glob
import os

# Finding all CSV files
csv_files = sorted(glob.glob('killed_in_gaza_batch_*.csv'))

# Creating a file to store the combined output
with open('killed_in_gaza.csv', 'w', encoding='utf-8-sig', newline='') as output_file:
    writer = None

    for filename in csv_files:
        with open(filename, 'r', encoding='utf-8-sig') as f:
            reader = csv.reader(f)
            
            if writer is None:
                headers = next(reader)  # read header
                writer = csv.writer(output_file)
                writer.writerow(headers)

            for row in reader:
                writer.writerow(row)
        os.remove(filename)


In [46]:
killed_in_gaza = Table.read_table('killed_in_gaza.csv')
killed_in_gaza.sort("Index", descending=True).show(5)

Index,Name,مس#ا,Born,Age,Sex,ID number
50021,Tamam Hussein Ouda Al-Batsh,شطبلا ةدوع 6سح مامت,1914-01-01,110,f,920861929
50020,Mariam Al-Abd Mahmoud Al-Kurd,دركلا دومحم دبعلا ميرم,1922-01-01,102,f,959190521
50019,Ahmed Mohammed Ibrahim Al-Tahrawi,يوارهطلا ميهاربا دمحم دمحأ,1922-01-01,101,m,940270614
50018,Ramadan Darwish Mustafa Rajab,بجر ىفطصم شيورد ناضمر,1924-01-01,100,m,906318118
50017,Jawdat Darwish Darwish Al-Maghribi,يبرغ{ا شيورد شيورد تدوج,1925-01-01,98,m,945227718


In [48]:
killed_in_gaza.num_rows

50020

In [66]:
import json
import csv

# Loading JSON
with open('infrastructure-damaged_2025-04-12.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Flattening records
flattened_data = []

for record in data:
    flat_record = {}
    for key, value in record.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flat_record[f"{key}_{sub_key}"] = sub_value
        else:
            flat_record[key] = value
    flattened_data.append(flat_record)

# Finding all possible headers from all records
all_keys = set()
for row in flattened_data:
    all_keys.update(row.keys())

# headers = sorted(all_keys)

# Saving to CSV
with open('infrastructure_damaged.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    
    for row in flattened_data:
        # filling missing keys with 0
        complete_row = {key: row.get(key, 0) for key in headers}
        writer.writerow(complete_row)


In [70]:
infrastructure_damaged = Table.read_table('infrastructure_damaged.csv')

# Making report_date the first column
labels = list(infrastructure_damaged.labels)
labels.remove('report_date')
new_order = ['report_date'] + labels  # Put it first

# Rearrange the Table
infrastructure_damaged = infrastructure_damaged.select(new_order)

infrastructure_damaged.show(5)

report_date,civic_buildings_destroyed,civic_buildings_ext_destroyed,educational_buildings_damaged,educational_buildings_destroyed,educational_buildings_ext_damaged,educational_buildings_ext_destroyed,places_of_worship_churches_destroyed,places_of_worship_ext_churches_destroyed,places_of_worship_ext_mosques_damaged,places_of_worship_ext_mosques_destroyed,places_of_worship_mosques_damaged,places_of_worship_mosques_destroyed,residential_destroyed,residential_ext_destroyed
2023-10-07,0,5,0,0,15,1,0,0,4,2,0,0,0,80
2023-10-08,0,11,0,0,30,1,0,0,8,4,0,0,159,159
2023-10-09,0,16,0,0,45,2,0,0,12,6,0,0,790,790
2023-10-10,0,22,0,0,60,2,0,0,17,8,0,0,1009,1009
2023-10-11,27,27,75,3,75,3,0,0,21,10,0,10,2835,2835


## 2. Exploratory Analysis & Visualization